<a href="https://colab.research.google.com/github/Pestrdan/netology_research/blob/main/Deep_learning_(Pytorch)/DLL_35_HW4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn
import torchvision as tv
from torchsummary import summary
import time

In [ ]:
import pandas as pd
results = pd.DataFrame(columns=['Модель', 'Train accuracy', 'Train Loss', 'Test accuracy', 'Epoch'])

In [ ]:
import gc
torch.cuda.empty_cache()
gc.collect()

0

In [ ]:
torch.cuda.empty_cache()
gc.collect()

0

In [ ]:
def evaluate_accuracy(data_iter, net):
    acc_sum, n = 0, 0
    net.eval()
    for X, y in data_iter:
        X, y = X.to(device), y.to(device)
        acc_sum += (net(X).argmax(axis=1) == y).sum()
        n += y.shape[0]
    return acc_sum.item() / n

In [ ]:
def train(model_name, net, train_iter, test_iter, trainer, num_epochs):
    net.to(device)
    loss = nn.CrossEntropyLoss(reduction='sum')
    net.train()
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()

        for i, (X, y) in enumerate(train_iter):
            X, y = X.to(device), y.to(device)
            trainer.zero_grad()
            y_hat = net(X)
            l = loss(y_hat, y)
            l.backward()
            trainer.step()
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
            n += y.shape[0]

            if i % 100 == 0:
              print(f"Step {i}. time since epoch: {time.time() -  start:.3f}. "
                    f"Train acc: {train_acc_sum / n:.3f}. Train Loss: {train_l_sum / n:.3f}")
        test_acc = evaluate_accuracy(test_iter, net.to(device))
        print('-' * 20)
        print(f'epoch {epoch + 1}, loss {train_l_sum / n:.4f}, train acc {train_acc_sum / n:.3f}'
              f', test acc {test_acc:.3f}, time {time.time() - start:.1f} sec')
        results.loc[len(results.index)] = [model_name, train_acc_sum/n, train_l_sum/n, test_acc, epoch+1]

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

# **ResNet 18**

In [ ]:
BATCH_SIZE = 256
transoforms = tv.transforms.Compose([
    tv.transforms.Grayscale(3),
    tv.transforms.Resize((224, 224)),
    tv.transforms.ToTensor()
])

train_dataset = tv.datasets.EMNIST('.', train=True, transform=transoforms, download=True, split='mnist')
test_dataset = tv.datasets.EMNIST('.', train=False, transform=transoforms, download=True, split='mnist')
train_iter = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE)
test_iter = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE)

100%|██████████| 561753746/561753746 [00:28<00:00, 19681525.43it/s]


Extracting ./EMNIST/raw/gzip.zip to ./EMNIST/raw


In [ ]:
model = tv.models.resnet18(pretrained=False)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [ ]:
#model

In [ ]:
#summary(model.to(device), input_size=(3, 224, 224))

In [ ]:
# Убираем требование градиента:
#for param in model.parameters():
    #param.requires_grad = False

In [ ]:
#model.fc

In [ ]:
model.fc = nn.Linear(in_features=512, out_features=10)

In [ ]:
#print("Params to learn:")
params_to_update = []
for name, param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        #print("\t",name)

In [ ]:
trainer = torch.optim.Adam(params_to_update, lr=0.001)

In [ ]:
train('ResNet_18', model, train_iter, test_iter, trainer, 1)

Step 0. time since epoch: 2.046. Train acc: 0.188. Train Loss: 2.352
Step 100. time since epoch: 46.332. Train acc: 0.942. Train Loss: 0.202
Step 200. time since epoch: 90.464. Train acc: 0.964. Train Loss: 0.126
--------------------
epoch 1, loss 0.1134, train acc 0.968, test acc 0.974, time 118.4 sec


In [ ]:
#results

# **VGG 16**

In [ ]:
import gc
torch.cuda.empty_cache()
gc.collect()

0

In [ ]:
torch.cuda.empty_cache()
gc.collect()

0

In [ ]:
BATCH_SIZE = 64
transoforms = tv.transforms.Compose([
    tv.transforms.Grayscale(3),
    tv.transforms.Resize((224, 224)),
    tv.transforms.ToTensor()
])

train_dataset = tv.datasets.EMNIST('.', train=True, transform=transoforms, download=True, split='mnist')
test_dataset = tv.datasets.EMNIST('.', train=False, transform=transoforms, download=True, split='mnist')
train_iter = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE)
test_iter = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [ ]:
model = tv.models.vgg16(pretrained=False)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [ ]:
#print(model)

In [ ]:
model.classifier[6] = nn.Linear(in_features=4096, out_features=10)

In [ ]:
#print("Params to learn:")
params_to_update = []
for name, param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        #print("\t",name)

In [ ]:
trainer = torch.optim.Adam(params_to_update, lr=0.001)

In [ ]:
train('VGG_16', model, train_iter, test_iter, trainer, 2)

Step 0. time since epoch: 0.268. Train acc: 0.125. Train Loss: 2.300
Step 100. time since epoch: 19.174. Train acc: 0.608. Train Loss: 2.634
Step 200. time since epoch: 37.931. Train acc: 0.781. Train Loss: 1.402
Step 300. time since epoch: 56.755. Train acc: 0.842. Train Loss: 0.979
Step 400. time since epoch: 75.504. Train acc: 0.874. Train Loss: 0.760
Step 500. time since epoch: 94.346. Train acc: 0.894. Train Loss: 0.628
Step 600. time since epoch: 113.215. Train acc: 0.908. Train Loss: 0.537
Step 700. time since epoch: 131.982. Train acc: 0.918. Train Loss: 0.471
Step 800. time since epoch: 150.790. Train acc: 0.925. Train Loss: 0.421
Step 900. time since epoch: 169.610. Train acc: 0.932. Train Loss: 0.382
--------------------
epoch 1, loss 0.3695, train acc 0.934, test acc 0.983, time 188.6 sec
Step 0. time since epoch: 0.198. Train acc: 0.984. Train Loss: 0.053
Step 100. time since epoch: 20.581. Train acc: 0.986. Train Loss: 0.051
Step 200. time since epoch: 40.879. Train acc: 

# **Inception v3**

In [ ]:
torch.cuda.empty_cache()
gc.collect()

0

In [ ]:
torch.cuda.empty_cache()
gc.collect()

0

In [ ]:
BATCH_SIZE = 64
transoforms = tv.transforms.Compose([
    tv.transforms.Grayscale(3),
    tv.transforms.Resize((299, 299)),
    tv.transforms.ToTensor()
])

train_dataset = tv.datasets.EMNIST('.', train=True, transform=transoforms, download=True, split='mnist')
test_dataset = tv.datasets.EMNIST('.', train=False, transform=transoforms, download=True, split='mnist')
train_iter = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE)
test_iter = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [ ]:
model = tv.models.inception_v3(pretrained=False)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/usr/local/lib/python3.10/dist-packages/torchvision/models/inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(


In [ ]:
model.aux_logits = False

In [ ]:
#print(model)

In [ ]:
model.fc = nn.Linear(in_features=2048, out_features=10)

In [ ]:
#print("Params to learn:")
params_to_update = []
for name, param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        #print("\t",name)

In [ ]:
trainer = torch.optim.Adam(params_to_update, lr=0.001)

In [ ]:
train('Inception_v3', model, train_iter, test_iter, trainer, 2)

Step 0. time since epoch: 0.510. Train acc: 0.078. Train Loss: 2.380
Step 100. time since epoch: 23.447. Train acc: 0.862. Train Loss: 0.432
Step 200. time since epoch: 46.282. Train acc: 0.915. Train Loss: 0.268
Step 300. time since epoch: 69.006. Train acc: 0.935. Train Loss: 0.207
Step 400. time since epoch: 91.669. Train acc: 0.945. Train Loss: 0.175
Step 500. time since epoch: 114.391. Train acc: 0.953. Train Loss: 0.152
Step 600. time since epoch: 137.072. Train acc: 0.958. Train Loss: 0.137
Step 700. time since epoch: 159.757. Train acc: 0.962. Train Loss: 0.124
Step 800. time since epoch: 182.427. Train acc: 0.965. Train Loss: 0.116
Step 900. time since epoch: 205.176. Train acc: 0.967. Train Loss: 0.110
--------------------
epoch 1, loss 0.1067, train acc 0.968, test acc 0.992, time 234.1 sec
Step 0. time since epoch: 0.271. Train acc: 1.000. Train Loss: 0.023
Step 100. time since epoch: 22.944. Train acc: 0.159. Train Loss: 3.377
Step 200. time since epoch: 45.761. Train acc:

In [ ]:
#results

# **DenseNet 161**

In [ ]:
torch.cuda.empty_cache()
gc.collect()

0

In [ ]:
torch.cuda.empty_cache()
gc.collect()

0

In [ ]:
BATCH_SIZE = 16
transoforms = tv.transforms.Compose([
    tv.transforms.Grayscale(3),
    tv.transforms.Resize((244, 244)),
    tv.transforms.ToTensor()
])

train_dataset = tv.datasets.EMNIST('.', train=True, transform=transoforms, download=True, split='mnist')
test_dataset = tv.datasets.EMNIST('.', train=False, transform=transoforms, download=True, split='mnist')
train_iter = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE)
test_iter = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [ ]:
model = tv.models.densenet161(pretrained=False)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [ ]:
model.classifier = nn.Linear(in_features=2208, out_features=10)

In [ ]:
#print("Params to learn:")
params_to_update = []
for name, param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        #print("\t",name)

In [ ]:
trainer = torch.optim.Adam(params_to_update, lr=0.001)

In [ ]:
train('Densenet_161', model, train_iter, test_iter, trainer, 2)

Step 0. time since epoch: 0.571. Train acc: 0.188. Train Loss: 2.335
Step 100. time since epoch: 12.219. Train acc: 0.746. Train Loss: 0.802
Step 200. time since epoch: 24.061. Train acc: 0.821. Train Loss: 0.571
Step 300. time since epoch: 35.781. Train acc: 0.855. Train Loss: 0.462
Step 400. time since epoch: 47.639. Train acc: 0.875. Train Loss: 0.399
Step 500. time since epoch: 59.423. Train acc: 0.889. Train Loss: 0.359
Step 600. time since epoch: 71.152. Train acc: 0.898. Train Loss: 0.326
Step 700. time since epoch: 83.009. Train acc: 0.906. Train Loss: 0.302
Step 800. time since epoch: 94.665. Train acc: 0.912. Train Loss: 0.283
Step 900. time since epoch: 106.604. Train acc: 0.918. Train Loss: 0.267
Step 1000. time since epoch: 118.393. Train acc: 0.922. Train Loss: 0.253
Step 1100. time since epoch: 130.416. Train acc: 0.927. Train Loss: 0.238
Step 1200. time since epoch: 142.388. Train acc: 0.930. Train Loss: 0.231
Step 1300. time since epoch: 154.091. Train acc: 0.932. Trai

# **Результаты**

In [ ]:
results

,Модель,Train accuracy,Train Loss,Test accuracy,Epoch
0,ResNet_18,0.967867,0.113362,0.9744,1
1,VGG_16,0.933617,0.369500,0.9834,1
2,VGG_16,0.984367,0.054867,0.9846,2
3,Inception_v3,0.967533,0.106704,0.9915,1
4,Inception_v3,0.105133,2.418758,0.1000,2
5,Densenet_161,0.959250,0.135996,0.9880,1
6,Densenet_161,0.981867,0.064082,0.9892,2


In [ ]:
results.sort_values(by = ['Test accuracy'], ascending=False)

,Модель,Train accuracy,Train Loss,Test accuracy,Epoch
3,Inception_v3,0.967533,0.106704,0.9915,1
6,Densenet_161,0.981867,0.064082,0.9892,2
5,Densenet_161,0.959250,0.135996,0.9880,1
2,VGG_16,0.984367,0.054867,0.9846,2
1,VGG_16,0.933617,0.369500,0.9834,1
0,ResNet_18,0.967867,0.113362,0.9744,1
4,Inception_v3,0.105133,2.418758,0.1000,2


**Выводы. В целом на относительно небольшом датасете все модели показали хороший результат даже на 1-2 эпохах. Там где было более одной эпохи, так же видим улучшение качества модели. Проблема возникла только с Inception v3. На старте она показала наилучшее качество, но за 1 первую эпоху очень сильно переобучилась и напротяжении всей второй эпохи только снижала качество**